In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp mfles

# MFLES model

In [ ]:
#| export
from itertools import cycle
from typing import Dict, List, Optional, Union

import numpy as np
import pandas as pd

In [ ]:
#| exporti
# utility functions
def cap_outliers(series, outlier_cap=3):
    mean = np.mean(series)
    std = np.std(series)
    return series.clip(
        min=mean - outlier_cap * std,
        max=mean + outlier_cap * std
    )

def calc_mse(y_true, y_pred):
    sq_err = (y_true - y_pred) ** 2
    return np.mean(sq_err)

def set_fourier(period):
    if period < 10:
        fourier = 5
    elif period < 70:
        fourier = 10
    else:
        fourier = 15
    return fourier

def calc_trend_strength(resids, deseasonalized):
    return max(0, 1 - (np.var(resids) / np.var(deseasonalized)))

def calc_seas_strength(resids, detrended):
    return max(0, 1 - (np.var(resids) / np.var(detrended)))

def calc_rsq(y, fitted):
    try:
        mean_y = np.mean(y)
        ssres = np.sum((y - fitted) ** 2)
        sstot = np.sum((y - mean_y) ** 2)
        return 1 - (ssres / sstot)
    except:
        return 0

def calc_cov(y, mult=1):
    if mult:
        # source http://medcraveonline.com/MOJPB/MOJPB-06-00200.pdf
        res = np.sqrt(np.exp(np.log(10)*(np.std(y)**2) - 1))
    else:
        res = np.std(y) / np.mean(y)
    return res

def get_seasonality_weights(y, seasonal_period):
    return 1 + np.arange(y.size) // seasonal_period

# feature engineering functions
def get_fourier_series(length, seasonal_period, fourier_order):
    x = 2 * np.pi * np.arange(1, fourier_order + 1) / seasonal_period
    t = np.arange(1, length + 1).reshape(-1, 1)
    x = x * t
    return np.hstack([np.cos(x), np.sin(x)])

def get_basis(y, n_changepoints, decay=-1, gradient_strategy=0):
    y = y.copy()
    y -= y[0]
    n = len(y)
    if gradient_strategy:
        gradients = np.abs(y[:-1] - y[1:])
    initial_point = y[0]
    final_point = y[-1]
    mean_y = np.mean(y)
    changepoints = np.empty(shape=(len(y), n_changepoints + 1))
    array_splits = []
    for i in range(1, n_changepoints + 1):
        i = n_changepoints - i + 1
        if gradient_strategy:
            cps = np.argsort(-gradients)
            cps = cps[cps > 0.1 * len(gradients)]
            cps = cps[cps < 0.9 * len(gradients)]
            split_point = cps[i-1]
            array_splits.append(y[:split_point])
        else:
            split_point = len(y)//i
            array_splits.append(y[:split_point])
            y = y[split_point:]
    len_splits = 0
    for i in range(n_changepoints):
        if gradient_strategy:
            len_splits = len(array_splits[i])
        else:
            len_splits += len(array_splits[i])
        moving_point = array_splits[i][-1]
        left_basis = np.linspace(initial_point, moving_point, len_splits)
        if decay is None:
            end_point = final_point
        else:
            if decay == -1:
                dd = moving_point**2 / (mean_y**2)
                if dd > 0.99:
                    dd = 0.99
                if dd < 0.001:
                    dd = 0.001
                end_point = moving_point - ((moving_point - final_point) * (1 - dd))
            else:
                end_point = moving_point - ((moving_point - final_point) * (1 - decay))
        right_basis = np.linspace(moving_point, end_point, n - len_splits + 1)
        changepoints[:, i] = np.append(left_basis, right_basis[1:])
    changepoints[:, i+1] = np.ones(n)
    return changepoints


def get_future_basis(basis_functions, forecast_horizon):
    n_components = np.shape(basis_functions)[1]
    slopes = np.gradient(basis_functions)[0][-1, :]
    future_basis = np.arange(0, forecast_horizon + 1)
    future_basis += len(basis_functions)
    future_basis = np.transpose([future_basis] * n_components)
    future_basis = future_basis * slopes
    future_basis = future_basis + (basis_functions[-1, :] - future_basis[0, :])
    return future_basis[1:, :]

def lasso_nb(X, y, alpha, tol=0.001, maxiter=10000):
    from sklearn.linear_model import Lasso

    lasso = Lasso(fit_intercept=False, tol=tol, max_iter=maxiter).fit(X, y) 
    return lasso.coef_

# different models
def siegel_repeated_medians(x,y):
    # Siegel repeated medians regression
    ys = y - y.reshape(-1, 1)
    xs = x - x.reshape(-1, 1)
    xs[xs == 0] = 1
    xs[np.diag_indices_from(xs)] = np.nan
    ys[xs == 0] = 0
    slopes = np.nanmedian(ys / xs, axis=1)
    ints = y - slopes * x
    return x * np.median(slopes) + np.median(ints)

def ses(x, alpha):
    return pd.Series(x).ewm(alpha=alpha, adjust=False).mean().to_numpy()

def ses_ensemble(y, min_alpha=0.05, max_alpha=1.0, smooth=False, order=1):
    #bad name but does either a ses ensemble or simple moving average    
    if smooth:
        results = np.zeros_like(y)
        alphas = np.arange(min_alpha, max_alpha, 0.05)
        for alpha in alphas:
            results += ses(y, alpha)
        results = results / len(alphas)
    else:
        results = pd.Series(y).rolling(order+1).mean().to_numpy()
        results[:order + 1] = y[:order + 1]
    return results

def fast_ols(x, y):
    """Simple OLS for two data sets."""
    M = x.size
    x_sum = x.sum()
    y_sum = y.sum()
    x_sq_sum = x @ x
    x_y_sum = x @ y
    slope = (M * x_y_sum - x_sum * y_sum) / (M * x_sq_sum - x_sum**2)
    intercept = (y_sum - slope * x_sum) / M
    return slope * x + intercept

def median(y, seasonal_period):
    if seasonal_period is None:
        return np.full_like(y, np.median(y))
    full_periods, resid = divmod(len(y), seasonal_period)
    period_medians = np.median(
        y[:full_periods * seasonal_period].reshape(full_periods, seasonal_period),
        axis=1,
    )
    medians = np.repeat(period_medians, seasonal_period)
    if resid:
        remainder_median = np.median(y[-seasonal_period:])
        medians = np.append(medians, np.repeat(remainder_median, resid))
    return medians

def ols(X, y):
    coefs = np.linalg.pinv(X.T.dot(X)).dot(X.T.dot(y))
    return X @ coefs

def wls(X, y, weights):
    weighted_X_T = X.T @ np.diag(weights)
    coefs = np.linalg.pinv(weighted_X_T.dot(X)).dot(weighted_X_T.dot(y))
    return X @ coefs

def _ols(X, y):
    return np.linalg.pinv(X.T.dot(X)).dot(X.T.dot(y))

class OLS:
    def fit(self, X, y):
        self.coefs = _ols(X, y)
    def predict(self, X):
        return X @ coefs

In [ ]:
#| export
class MFLES:
    def __init__(self, verbose=1, robust=None):
        self.penalty = None
        self.trend = None
        self.seasonality = None
        self.robust = robust
        self.const = None
        self.aic = None
        self.upper = None
        self.lower= None
        self.exogenous_models = None
        self.verbose = verbose
        self.predicted = None

    def fit(self,
            y,
            seasonal_period=None,
            X=None,
            fourier_order=None,
            ma=None,
            alpha=.1,
            decay=-1,
            n_changepoints=.25,
            seasonal_lr=.9,
            rs_lr=1,
            exogenous_lr=1,
            exogenous_estimator=OLS,
            exogenous_params={},
            linear_lr=.9,
            cov_threshold=.7,
            moving_medians=False,
            max_rounds=50,
            min_alpha=.05,
            max_alpha=1.0,
            round_penalty=0.0001,
            trend_penalty=True,
            multiplicative=None,
            changepoints=True,
            smoother=False,
            seasonality_weights=False):
        """
        

        Parameters
        ----------
        y : TYPE
            DESCRIPTION.
        seasonal_period : TYPE, optional
            DESCRIPTION. The default is None.
        fourier_order : TYPE, optional
            DESCRIPTION. The default is None.
        ma : TYPE, optional
            DESCRIPTION. The default is None.
        alpha : TYPE, optional
            DESCRIPTION. The default is .1.
        decay : TYPE, optional
            DESCRIPTION. The default is -1.
        n_changepoints : TYPE, optional
            DESCRIPTION. The default is .25.
        seasonal_lr : TYPE, optional
            DESCRIPTION. The default is .9.
        rs_lr : TYPE, optional
            DESCRIPTION. The default is 1.
        linear_lr : TYPE, optional
            DESCRIPTION. The default is .9.
        cov_threshold : TYPE, optional
            DESCRIPTION. The default is .7.
        moving_medians : TYPE, optional
            DESCRIPTION. The default is False.
        max_rounds : TYPE, optional
            DESCRIPTION. The default is 10.
        min_alpha : TYPE, optional
            DESCRIPTION. The default is .05.
        max_alpha : TYPE, optional
            DESCRIPTION. The default is 1.0.
        trend_penalty : TYPE, optional
            DESCRIPTION. The default is True.
        multiplicative : TYPE, optional
            DESCRIPTION. The default is None.
        changepoints : TYPE, optional
            DESCRIPTION. The default is True.
        smoother : TYPE, optional
            DESCRIPTION. The default is False.

        Returns
        -------
        None.

        """
        if cov_threshold == -1:
            cov_threshold = 10000
        n = len(y)
        if n < 4 or np.all(y == np.mean(y)):
            if self.verbose:
                if n < 4:
                    print('series is too short (<4), defaulting to naive')
                else:
                    print(f'input is constant with value {y[0]}, defaulting to naive')
            self.trend = np.append(y[-1], y[-1])
            self.seasonality = np.zeros(len(y))
            self.trend_penalty = False
            self.mean = 0
            self.std = 0
            return np.tile(y[-1], len(y))
        og_y = y
        self.og_y = og_y
        y = y.copy()
        if n_changepoints is None:
            changepoints = False
        if isinstance(n_changepoints, float) and n_changepoints < 1:
            n_changepoints = int(n_changepoints * n)
        self.linear_component = np.zeros(n)
        self.seasonal_component = np.zeros(n)
        self.ses_component = np.zeros(n)
        self.median_component = np.zeros(n)
        self.exogenous_component = np.zeros(n)
        self.exogenous_lr = exogenous_lr
        self.exo_model = []
        self.round_cost = []
        if multiplicative is None:
            if seasonal_period is None:
                multiplicative = False
            else:
                multiplicative = True
            if min(y) <= 0:
                multiplicative = False
        if multiplicative:
            self.const = y.min()
            y = np.log(y)
        else:
            self.const = None
            self.std = np.std(y)
            self.mean = np.mean(y)
            y = (y - self.mean) / self.std
        if seasonal_period is not None:
            if not isinstance(seasonal_period, list):
                seasonal_period = [seasonal_period]
        self.trend_penalty = trend_penalty
        if moving_medians and seasonal_period is not None:
            fitted = median(y, max(seasonal_period))
        else:
            fitted = median(y, None)
        self.median_component += fitted
        self.trend = np.append(fitted.copy()[-1:], fitted.copy()[-1:])
        mse = None
        equal = 0
        if ma is None:
            ma_cycle = cycle([1])
        else:
            if not isinstance(ma, list):
                ma = [ma]
            ma_cycle = cycle(ma)
        if seasonal_period is not None:
            seasons_cycle = cycle(list(range(len(seasonal_period))))
            self.seasonality = np.zeros(max(seasonal_period))
            fourier_series = []
            for period in seasonal_period:
                if fourier_order is None:
                    fourier = set_fourier(period)
                else:
                    fourier = fourier_order
                fourier_series.append(get_fourier_series(n,
                                                    period,
                                                    fourier))
            if seasonality_weights:
                cycle_weights = []
                for period in seasonal_period:
                    cycle_weights.append(get_seasonality_weights(y, period))
        else:
            self.seasonality = None
        for i in range(max_rounds):
            resids = y - fitted
            if mse is None:
                mse = calc_mse(y, fitted)
            else:
                if mse <= calc_mse(y, fitted):
                    if equal == 6:
                        break
                    equal += 1
                else:
                    mse = calc_mse(y, fitted)
                self.round_cost.append(mse)
            if seasonal_period is not None:
                seasonal_period_cycle = next(seasons_cycle)
                if seasonality_weights:
                    seas = wls(fourier_series[seasonal_period_cycle],
                               resids,
                               cycle_weights[seasonal_period_cycle])
                else:
                    seas = ols(fourier_series[seasonal_period_cycle],
                               resids)
                seas = seas * seasonal_lr
                component_mse = calc_mse(y, fitted + seas)
                if mse > component_mse:
                    mse = component_mse
                    fitted += seas
                    resids = y - fitted
                    self.seasonality += np.resize(seas[-seasonal_period[seasonal_period_cycle]:],
                                                  len(self.seasonality))
                    self.seasonal_component += seas
            if X is not None and i > 0:
                model_obj = exogenous_estimator(**exogenous_params)
                model_obj.fit(X, resids)
                self.exo_model.append(model_obj)
                _fitted_values = model_obj.predict(X) * exogenous_lr
                self.exogenous_component += _fitted_values
                fitted += _fitted_values
                resids = y - fitted
            if i % 2: #if even get linear piece, allows for multiple seasonality fitting a bit more
                if self.robust:
                    tren = siegel_repeated_medians(x=np.arange(n), y=resids)
                else:
                    if i==1 or not changepoints:
                        tren = fast_ols(x=np.arange(n),
                                        y=resids)
                    else:
                        cps = min(n_changepoints, int(.1*n))
                        lbf = get_basis(y=resids,
                                        n_changepoints=cps,
                                        decay=decay)
                        tren = np.dot(lbf, lasso_nb(lbf, resids, alpha=alpha))
                        tren = tren * linear_lr
                component_mse = calc_mse(y, fitted + tren)
                if mse > component_mse:
                    mse = component_mse
                    fitted += tren
                    self.linear_component += tren
                    self.trend += tren[-2:]
                    if i == 1:
                        self.penalty = calc_rsq(resids, tren)
            elif i > 4 and not i % 2:
                if smoother is None:
                    if seasonal_period is not None:
                        len_check = int(max(seasonal_period))
                    else:
                        len_check = 12
                    if resids[-1] > np.mean(resids[-len_check:-1]) + 3 * np.std(resids[-len_check:-1]):
                        smoother = 0
                    if resids[-1] < np.mean(resids[-len_check:-1]) - 3 * np.std(resids[-len_check:-1]):
                        smoother = 0
                    if resids[-2] > np.mean(resids[-len_check:-2]) + 3 * np.std(resids[-len_check:-2]):
                        smoother = 0
                    if resids[-2] < np.mean(resids[-len_check:-2]) - 3 * np.std(resids[-len_check:-2]):
                        smoother = 0
                    if smoother is None:
                        smoother = 1
                    else:
                        resids[-2:] = cap_outliers(resids, 3)[-2:]
                tren = ses_ensemble(resids,
                                    min_alpha=min_alpha,
                                    max_alpha=max_alpha,
                                    smooth=smoother*1,
                                    order=next(ma_cycle)
                                    )
                tren = tren * rs_lr
                component_mse = calc_mse(y, fitted + tren)
                if mse > component_mse + round_penalty * mse:
                    mse = component_mse
                    fitted += tren
                    self.ses_component += tren
                    self.trend += tren[-1]
            if i == 0: #get deasonalized cov for some heuristic logic
                if self.robust is None:
                    try:
                        if calc_cov(resids, multiplicative) > cov_threshold:
                            self.robust = True
                        else:
                            self.robust = False
                    except:
                        self.robust = True

            if i == 1:
                resids = cap_outliers(resids, 5) #cap extreme outliers after initial rounds
        if multiplicative:
            fitted = np.exp(fitted)
        else:
            fitted = self.mean + (fitted * self.std)
        self.multiplicative = multiplicative
        self.fitted = fitted
        return fitted

    def predict(self, forecast_horizon, X=None):
        last_point = self.trend[1]
        slope = last_point - self.trend[0]
        if self.trend_penalty and self.penalty is not None:
            slope = slope * max(0, self.penalty)
        self.predicted_trend = slope * np.arange(1, forecast_horizon + 1) + last_point
        if self.seasonality is not None:
            predicted = self.predicted_trend + np.resize(self.seasonality, forecast_horizon)
        else:
            predicted = self.predicted_trend
        if X is not None:
            for model in self.exo_model:
                predicted += model.predict(X) * self.exogenous_lr
        if self.const is not None:
            predicted = np.exp(predicted)
        else:
            predicted = self.mean + (predicted * self.std)
        self.predicted = predicted
        return predicted

In [ ]:
#| hide
url = "https://raw.githubusercontent.com/tidyverts/tsibbledata/master/data-raw/vic_elec/VIC2015/demand.csv"
df = pd.read_csv(url)
df["Date"] = df["Date"].apply(
    lambda x: pd.Timestamp("1899-12-30") + pd.Timedelta(x, unit="days")
)
df["ds"] = df["Date"] + pd.to_timedelta((df["Period"] - 1) * 30, unit="m")
timeseries = df[["ds", "OperationalLessIndustrial"]]
timeseries.columns = [
    "ds",
    "y",
]  # Rename to OperationalLessIndustrial to y for simplicity.

# Filter for first 149 days of 2012.
start_date = pd.to_datetime("2012-01-01")
end_date = start_date + pd.Timedelta("149D")
mask = (timeseries["ds"] >= start_date) & (timeseries["ds"] < end_date)
timeseries = timeseries[mask]

# Resample to hourly
timeseries = timeseries.set_index("ds").resample("H").sum()
timeseries.head()

# decomposition
mfles = MFLES()
fitted = mfles.fit(y=timeseries.y.values, seasonal_period=[24, 24 * 7])
predicted = mfles.predict(forecast_horizon=24)